In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import vpython as vp

%matplotlib inline

# Jupyter LabでVPythonを動かすためには
# % pip3 install jupyterlab-vpython
# が必要だった。

In [ ]:
b = vp.box()

In [ ]:
b.color = vp.vector(1,1,0)

In [ ]:
b.size = vp.vector(1,2,5)

In [ ]:
b.pos = vp.vector(2,0,-5)

In [ ]:
# キャンバスの作成
scene = vp.canvas(width=600, height=300, title='Cube-Spring') # Enable to restart
# 3次元空間内の原点を見るために，球を原点に配置。
orig = vp.sphere(pos=vp.vector(0,0,0), radius=0.1, color=vp.color.red)

In [ ]:
# オブジェクトの作成
# 立方体
cube_size = 1
cube = vp.box(size=vp.vector(cube_size, cube_size, cube_size), color=vp.color.orange)
cube.pos = vp.vector(0, cube.height/2, 0)

In [ ]:
# 床
floor = vp.box(length=5.0, height=0.1, width=cube_size+0.2, color=vp.color.green)
floor.pos = vp.vector(0, -floor.height/2, 0)

In [ ]:
# 壁
wall  = vp.box(length=0.1, height=1.5, width=floor.width, color=vp.color.yellow)
wall.pos = vp.vector((-wall.length/2-floor.length/2), (wall.height/2 - floor.height) , 0)

In [ ]:
#equi_length = box_size/2+floor.height/2 #バネの自然長
#wall_surface_pos = cube_size/2+floor.height/2

In [ ]:
# バネ
spring_pos_wall = vp.vector((wall.pos.x+wall.length/2), cube.pos.y, cube.pos.z )
spring_pos_cube = vp.vector( (cube.pos.x-cube.length/2), cube.pos.y, cube.pos.z)
spring = vp.helix(pos=spring_pos_wall, axis=(spring_pos_cube - spring_pos_wall), 
                  radius=0.2,     # バネ径の半径
                  thickness=0.05, # バネ寸法
                  coils=8,        # バネ巻数
                  color=vp.vector(0, 1, 1) # cyan
                 )

In [ ]:
def func_pos(k):
    return np.sin(k*0.1)

for k in range(100):
    vp.sleep(0.1)
    x_pos = func_pos(k)
    cube.pos = vp.vector(x_pos, cube.pos.y, 0)
    spring_pos_cube = vp.vector( (cube.pos.x-cube.length/2), cube.pos.y, cube.pos.z)
    spring.axis = spring_pos_cube - spring_pos_wall

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import vpython as vp

%matplotlib inline

In [ ]:
# Create Scene
scene = vp.canvas(width=600, height=400, title='Animation') # Enable to restart
scene.camera.pos  = vp.vector(0, 6, 10)
scene.camera.axis = vp.vector(0, -2, -8) - scene.camera.pos

In [ ]:
# 床の作成，床の真ん中を原点とする
floor = vp.box(length=40, height=0.8, width=60, color=vp.color.green)
floor.pos = vp.vector( 0, -(floor.height/2), -floor.width/2)

In [ ]:
# 球の作成
ball_radius = 0.5 # 半径

# 的球の初期位置のx座標
b2_x_init = 2*ball_radius - ball_radius  # theta2 -> 30 [deg]
#b2_x_init = 2*ball_radius - 0.59*ball_radius # theta2 -> 45 [deg]
#b2_x_init = 2*ball_radius - 0.0076*ball_radius # almost theta2 -> 85 [deg]

# 手球b1, 的球b2
b1 = vp.sphere(pos=vp.vector(0, ball_radius, 0), radius=ball_radius, \
               color=vp.color.white)
b2 = vp.sphere(pos=vp.vector(b2_x_init, ball_radius, -8), radius=ball_radius, \
               color=vp.color.red)

In [ ]:
# 各パラメータ
v1, v2 = 0.4, 0   # 初期速度
theta1, theta2=0, 0 # 初期角度
c_rest = 0.8 # 反発係数，coefficient of restitution
flag = False  # 衝突判定フラグ，Trueは衝突検知

In [ ]:
def ball_step(ball, v, theta):
    coef1 = 1
    x = coef1*v*np.sin(theta)
    z = -coef1*v*np.cos(theta) # 奥行きがマイナスｚ軸方向ゆえ"-"がつく
    ball.pos += vp.vector(x, 0, z)

In [ ]:
def check_collision():
#def check_collision(b1, b2, flag):
    global b1, b2, flag, v1, v2, theta1, theta2, c_rest
    dx = b1.pos.x - b2.pos.x
    dz = b1.pos.z - b2.pos.z
    d = np.sqrt(dx*dx + dz*dz)
    if d <= (b1.radius + b2.radius):
        flag = True
        theta2 = np.arcsin(-(b1.pos.x-b2.pos.x)/(2*b1.radius) )
        theta1 = theta2 - np.pi/2
        v2 = 0.5*v1*(1.0 + c_rest)
        v1 = 0.5*v1*(1.0 - c_rest)

In [ ]:
for k in range(100):
    vp.sleep(0.01)
    ball_step(b1, v1, theta1)
    ball_step(b2, v2, theta2)
    if not flag:
        check_collision()

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import vpython as vp
import matplotlib.pyplot as plt 
from scipy.integrate import odeint
from scipy.special import ellipk

%matplotlib inline

In [ ]:
def dFunc(x, time, string_len, g):
    dx0 = x[1]
    dx1 = -(g/string_len)*np.sin(x[0]) # string_len:糸の長さ
    return [dx0, dx1]

In [ ]:
string1_len = 10
theta1_ini = 20*(np.pi/180) # deg -> rad
v1_ini= 0

string2_len = 10
theta2_ini = 60*(np.pi/180) # deg -> rad
v2_ini = v1_ini

In [ ]:
g = 9.80665 # 重力加速度
#time=np.linspace(0,6.9,100)
time = np.arange(0, 10, 0.01)
sol_1 = odeint(dFunc, [theta1_ini, v1_ini], time, args=(string1_len,g,))
sol_2 = odeint(dFunc, [theta2_ini, v2_ini], time, args=(string2_len,g,))

In [ ]:
print(len(sol_1))
print(sol_1.ndim)
print(sol_1.shape)
print(sol_1.size)

In [ ]:
plt.plot(time, sol_1[:,0])
plt.plot(time, sol_2[:,0])
plt.legend(["string1", "string2"])
plt.xlabel("time")
plt.ylabel("theta [rad]")

In [ ]:
# Create Scene
scene = vp.canvas(width=600, height=400, title='Pendulum Animation') # Enable to restart
scene.camera.pos  = vp.vector(0, 6, 20)
scene.camera.axis = vp.vector(0, 2, -100) - scene.camera.pos


In [ ]:
floor = vp.box(pos=vp.vector(0,0,0),length=40, height=0.1, width=60, color=vp.color.green)
#floor.pos = vp.vector( 0, -(floor.height/2), floor.width/2) # 床の端の真ん中を原点とする
#print(floor.pos)

In [ ]:
origin = vp.sphere(pos=vp.vector(0, 0, 0), radius=0.5, color=vp.color.red) # show original point of 3D space
#org    = vp.sphere(pos=vp.vector(1, 0, 1), radius=0.5, color=vp.color.cyan) # exam. position

In [ ]:
bar_height = string1_len+2.0
bar = vp.cylinder(pos=vp.vector(0, bar_height, 0), axis=vp.vector(0,0,-10), radius=0.5, color=vp.color.white)

In [ ]:
node1 = vp.vector(0, bar.pos.y, 0)
end1 = vp.vector( string1_len*np.sin(theta1_ini),  bar.pos.y-string1_len*np.cos(theta1_ini), 0)
str1 = vp.cylinder(pos=node1, axis=-(node1-end1), radius=0.02, color=vp.color.yellow)
#m1 = vp.sphere(pos=end1, radius=1, color=vp.color.white)
m1 = vp.cone(pos=end1, axis=vp.vector(0, -1, 0), radius=0.5, color=vp.color.white)

In [ ]:
node2 = vp.vector(0, bar.pos.y, -5)
end2 = vp.vector( string2_len*np.sin(theta2_ini),  bar.pos.y-string2_len*np.cos(theta2_ini), node2.z)
str2 = vp.cylinder(pos=node2, axis=-(node2-end2), radius=0.02, color=vp.color.yellow)
#m2 = vp.sphere(pos=end2, radius=1, color=vp.color.white)
m2 = vp.cone(pos=end2, axis=vp.vector(0, -1, 0), radius=0.5, color=vp.color.white)

In [ ]:
tstart, tend = 0.0, 0.0
dt = 0.1
nstep = 5
theta1 = theta1_ini
v1 = v1_ini

theta2 = theta2_ini
v2 = v2_ini

vp.sleep(3)

for k in range(200):
    vp.sleep(0.01)
    tstart = k*dt
    tend = (k+1)*dt
    time = np.linspace(tstart, tend, nstep)
    sol_1 = odeint(dFunc, [theta1, v1], time, args=(string1_len,g,))
    sol_2 = odeint(dFunc, [theta2, v2], time, args=(string2_len,g,))

    #    print(k,sol_1[k][0])
    theta1 = sol_1[nstep-1][0] # angle
    v1     = sol_1[nstep-1][1] # angular velocity

    theta2 = sol_2[nstep-1][0]
    v2     = sol_2[nstep-1][1]
#    print(k,theta1)
    end1 = vp.vector( string1_len*np.sin(theta1),  bar.pos.y-string1_len*np.cos(theta1), 0)
#    print(end1)
    str1.axis = -(node1-end1)
    m1.pos = end1
    
    end2 = vp.vector( string2_len*np.sin(theta2),  bar.pos.y-string2_len*np.cos(theta2), node2.z)
    str2.axis =-(node2-end2)
    m2.pos = end2  

In [ ]:
deg_list = np.arange(5, 95, 5) # 0 - 90, 5 deg step
periodT = np.zeros(np.size(deg_list))
k = 0
for deg in deg_list:
    periodT[k] = (2/np.pi)*ellipk( np.sin( (deg*np.pi/180)/2. )**2 )
    print(k, deg, periodT[k])
    k += 1

In [ ]:
plt.plot(deg_list, periodT, marker='x')
plt.xlabel('theta_M [deg]')
plt.ylabel('T')
plt.grid()
#plt.savefig('fig_Anima_Pendulum_Analysis.png')

plt.show()